In [ ]:
import pandas as pd
import google.generativeai as genai
from pandasai.llm.base import LLM
from pandasai import SmartDataframe
from typing import Optional
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")

class GeminiFlash(LLM):
    def __init__(self, api_key: str, model: str = "gemini-2.5-flash"):
        self.api_key = api_key
        self.model_name = model
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel(model_name=model)

    def call(self, instruction: str, context: Optional[object] = None, **kwargs) -> str:
        if isinstance(context, list):
            full_prompt = "\n".join(context) + "\n\n" + instruction
        elif isinstance(context, dict):
            full_prompt = str(context) + "\n\n" + instruction
        else:
            full_prompt = instruction

        try:
            response = self.model.generate_content(str(full_prompt))
            result = response.text if hasattr(response, "text") else str(response)
            return result
        except Exception as e:
            print(f"Gemini Error: {e}")
            return str(e)

    @property
    def type(self) -> str:
        return "gemini-flash"

df_train = pd.read_csv('data/train.csv')
df_train['Gender_num'] = df_train['Gender'].map({'Male': 0, 'Female': 1})
df_gender = df_train[['Gender_num']].copy()


llm = GeminiFlash(api_key)

sdf = SmartDataframe(df_gender, config={
    "llm": llm,
    "enable_cache": False
})

try:
    result = sdf.chat("plot Gender_num ( 0 is Male, 1 is Female) ")
    print("Answer:", result)
except Exception as e:
    print("PandasAI crash:", e)
